In [1]:
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import pathlib as pl
import shutil
import sys

import flopy
from xmipy import XmiWrapper
from modflowapi import ModflowApi

from bmi.wrapper import BMIWrapper

/home/parallels/miniconda3/envs/liss/lib/python3.10/site-packages/bmi/__init__.py:2: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
/home/parallels/miniconda3/envs/liss/lib/python3.10/site-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/home/parallels/miniconda3/envs/liss/lib/python3.10/site-packages/pkg_resources/__init__.py:3121: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydap.responses')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/refere

In [2]:
sys.path.append("../common")
from liss_settings import libmf6, get_dflow_grid_name, get_sfincs_grid_name, get_sfincs_dtuser, get_modflow_coupling_tag

In [3]:
coastal_name = "sfincs"

In [4]:
control_path = pl.Path("../sfincs/test_north_fork/run/sfincs.inp") # change this if using a different D-Flow FM control file
grid_name = get_sfincs_grid_name(control_path)
print(grid_name)

sfincs


In [5]:
coastal_dtuser = get_sfincs_dtuser(control_path)
print(coastal_dtuser)

300.0


#### Set unit conversion factors

In [6]:
d2sec = 24. * 60. * 60.
hrs2sec = 60. * 60. 
m2ft = 3.28081
cfd2cms = 1.0 / ((3.28082**3) * 86400.)

#### Set the MODFLOW coupling frequency

Change the `mf_couple_freq_hours` value. Only tested for multiple of the D-Flow FM DtUser variable. Will not work for `mf_couple_freq_hours` values greater than 24.

In [7]:
mf_couple_freq_hours = 1.0  #Change this value to change the coupling frequency
mf_couple_freq = mf_couple_freq_hours * hrs2sec
coastal_per_mf = int(mf_couple_freq / coastal_dtuser)

In [8]:
print(f"MODFLOW coupling frequency {mf_couple_freq_hours} hours\nMODFLOW coupled to the coastal every {coastal_per_mf} output time step ({coastal_dtuser} sec.)") 

MODFLOW coupling frequency 1.0 hours
MODFLOW coupled to the coastal every 12 output time step (300.0 sec.)


In [9]:
mf_tag = get_modflow_coupling_tag(mf_couple_freq_hours)
print(f"MODFLOW coupling tag: {mf_tag}")

MODFLOW coupling tag: 01.00H


In [10]:
nstp = int(86400.0 / (coastal_per_mf * coastal_dtuser))
print(f"MODFLOW time steps per day: {nstp}")

MODFLOW time steps per day: 24


#### Set a few variables for controlling coupling

In [11]:
HDRY = -1e30
DEPTH_MIN = 0.1

#### Print the path of the modflow6 shared library

In [12]:
str(libmf6), libmf6.is_file()

('/home/parallels/miniconda3/envs/liss/lib/libmf6.so', True)

#### Load the D-FLOW to MODFLOW weights

In [13]:
npzfile = np.load(f"../mapping/{coastal_name}2mfghb_{grid_name}.npz")
npzfile

NpzFile '../mapping/sfincs2mfghb_sfincs.npz' with keys: sfincs2mfghb, ghbmask, ghb2qext

In [14]:
sfincs2mfghb = npzfile[f"{coastal_name}2mfghb"]
ghbmask = npzfile["ghbmask"]
ghb2qext = npzfile["ghb2qext"]

#### Define paths for the model simulation

In [15]:
mf_base_path = pl.Path("../modflow/greenport500ft/base/").resolve()
mf_run_path = pl.Path(f"../modflow/greenport500ft/run_{mf_tag}_sfincs/").resolve()
print(mf_run_path)

/media/psf/Documents/Work/compound_flooding/modflow/greenport500ft/run_01.00H_sfincs


#### Load the base MODFLOW model

In [16]:
sim = flopy.mf6.MFSimulation.load(sim_ws=mf_base_path)
gwf = sim.get_model()

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package ic...
    loading package npf...
    loading package obs...
    loading package sto...
    loading package oc...
    loading package ghb...
    loading package wel...
    loading package drn...
    loading package rch...
  loading solution package modflow...


In [17]:
sim.set_sim_path(mf_run_path)

In [18]:
gwf = sim.get_model()

In [19]:
cell_area = gwf.dis.delr.array[0] * gwf.dis.delc.array[0] / (m2ft * m2ft)
cell_area

23226.183272035516

#### Change TDIS to hourly time steps

In [20]:
tdis = sim.get_package("TDIS")
perioddata = tdis.perioddata.array

In [21]:
perioddata["nstp"] = nstp
tdis.perioddata = perioddata

#### Write the new model files

In [22]:
sim.write_simulation()

writing simulation...
  writing simulation name file...
  writing simulation tdis package...
  writing solution package modflow...
  writing model modflow...
    writing model name file...
    writing package dis...
    writing package ic...
    writing package npf...
    writing package obs-1...
    writing package sto...
    writing package oc...
    writing package ghb_obs...
    writing package ghb-1...
    writing package wel-1...
    writing package drn_obs...
    writing package drn-1...
    writing package rch-1...


#### Define base GHB variables

In [23]:
ghb_data0 = gwf.ghb.stress_period_data.get_dataframe()[0]
ghb_data0

,cellid_layer,cellid_row,cellid_column,bhead,cond,iface,boundname
0,0,26,0,0.0,24999.999994,0,'sound'
1,0,27,0,0.0,25000.000000,0,'sound'
2,0,28,0,0.0,24999.999994,0,'sound'
3,0,29,0,0.0,24231.956659,0,'sound'
4,0,25,1,0.0,25000.000006,0,'sound'
...,...,...,...,...,...,...,...
730,0,14,56,0.0,22494.219708,0,'peconic'
731,0,15,56,0.0,25000.000006,0,'peconic'
732,0,16,56,0.0,24999.999994,0,'peconic'
733,0,17,56,0.0,25000.000000,0,'peconic'


#### Setup and initialize SFINCS

You will need to set `coastal_dirpath` to the correct directory on your machine.

In [24]:
# dflow_dirpath = os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\dflowfm\bin")
# dflow_deps_dirpath = (
#     os.path.abspath(r"C:\Program Files\Deltares\Delft3D FM Suite 2023.02 HM\plugins\DeltaShell.Dimr\kernels\x64\share\bin"),
# )
coastal_dirpath = pl.Path("/usr/local/lib/")
coastal_lib = "libsfincs.so"
sys.path.append(coastal_dirpath)
libcoastal = coastal_dirpath / coastal_lib

In [25]:
nb_path = pl.Path(".").resolve().parent

coastal_base = nb_path / "sfincs/test_north_fork/base"
coastal_working = nb_path / "sfincs/test_north_fork/run"
coastal_config = coastal_working / "sfincs.inp"  # change this for a different coastal model

In [26]:
if coastal_working.is_dir():
    shutil.rmtree(coastal_working)
shutil.copytree(coastal_base, coastal_working)

PosixPath('/media/psf/Documents/Work/compound_flooding/sfincs/test_north_fork/run')

In [27]:
# Add dflowfm dll folder to PATH so that it can be found by the BMIWrapper
os.environ["PATH"] = (
    str(coastal_dirpath) + os.pathsep + os.environ["PATH"]
)

In [28]:
(pl.Path(coastal_dirpath) / coastal_lib).is_file()

True

#### Initialize the coastal model

In [29]:
coastal = XmiWrapper(libcoastal, working_directory=coastal_working)

In [30]:
coastal.initialize()


 ----------- Welcome to SFINCS -----------

  @@@@@  @@@@@@@ @@ @@  @@   @@@@   @@@@@ 
 @@@ @@@ @@@@@@@ @@ @@@ @@ @@@@@@@ @@@ @@@
 @@@     @@      @@ @@@ @@ @@   @@ @@@    
  @@@@@  @@@@@@  @@ @@@@@@ @@       @@@@@ 
     @@@ @@      @@ @@ @@@ @@   @@     @@@
 @@@ @@@ @@      @@ @@  @@  @@@@@@ @@@ @@@
  @@@@@  @@      @@ @@   @   @@@@   @@@@@ 

              ..............              
          ......:@@@@@@@@:......          
       ..::::..@@........@@.:::::..       
     ..:::::..@@..::..::..@@.::::::..     
    .::::::..@@............@@.:::::::.    
   .::::::..@@..............@@.:::::::.   
  .::::::::..@@............@@..::::::::.  
 .:::::::::...@@.@..@@..@.@@..::::::::::. 
 .:::::::::...:@@@..@@..@@@:..:::::::::.. 
 ............@@.@@..@@..@@.@@............ 
 ^^^~~^^~~^^@@..............@@^^^~^^^~~^^ 
 .::::::::::@@..............@@.:::::::::. 
  .......:.@@.....@.....@....@@.:.......  
   .::....@@......@.@@@.@....@@.....::.   
    .:::~@@.:...:.@@...@@.:.:.@@~::::.    
     .::

#### Get data from the coastal model

This is specific to the coastal model being used.

In [31]:
# stage
zs = coastal.get_value_ptr("zs")
print("min/max water levels: ", min(zs), max(zs))
print("water level dimensions: ", len(zs))

# bottom
zb = coastal.get_value_ptr("zb")
print("min/max bottom: ", zb.min(), zb.max())

# inflow array
qexchange = coastal.get_value_ptr("qext")
print("min/max qexchange: ", qexchange.min(), qexchange.max())
print("exchange dimensions: ", qexchange.shape[0])


min/max water levels:  0.0 156.7672
water level dimensions:  14048
min/max bottom:  -82.27512 156.7672
min/max qexchange:  0.0 0.0
exchange dimensions:  14048


#### Initialize MODFLOW using MODFLOW API

In [32]:
mf6 = ModflowApi(str(libmf6), working_directory=mf_run_path)

In [33]:
mf6.initialize()

#### Define MODFLOW variable tags and set pointer to MODFLOW variables

In [34]:
ghb_bhead_tag = mf6.get_var_address("BHEAD", "MODFLOW", "GHB-1")
ghb_cond_tag = mf6.get_var_address("COND", "MODFLOW", "GHB-1")
ghb_flow_tag = mf6.get_var_address("SIMVALS", "MODFLOW", "GHB-1")

In [35]:
ghb_bhead_ptr = mf6.get_value_ptr(ghb_bhead_tag)
ghb_cond_ptr = mf6.get_value_ptr(ghb_cond_tag)
ghb_flow = np.zeros(ghb_bhead_ptr.shape)

#### Create dictionaries for saving modified GHB data

In [36]:
elev_dict = {}
cond_dict = {}
qext_dict = {}

#### Function to update MODFLOW GHB data

This is specific to the coastal model being used.

In [37]:
def update_mf(key, s_ptr, z):
    s = s_ptr.copy()
    d = s - z
    mask = d == 0.0
    s[mask] = 0.0
    mult = np.full(d.shape, 1.0)
    mult[mask] = 0.0

    ghb_head = ghb_data0["bhead"].to_numpy()
    ghb_head[ghbmask] = sfincs2mfghb.dot(s)[ghbmask] * m2ft
    ghb_cond = ghb_data0["cond"].to_numpy()
    ghb_cond[ghbmask] = ghb_cond[ghbmask] * sfincs2mfghb.dot(mult)[ghbmask]
    
    ghb_bhead_ptr[:] = ghb_head[:]
    ghb_cond_ptr[:] = ghb_cond[:]

    # update results dictionary
    elev_dict[key] = ghb_head.copy()
    cond_dict[key] = ghb_cond.copy()

#### Function to update Qext data in the coastal model

This is specific to the coastal model being used.

In [38]:
def update_coastal(key, dt, s_ptr, z):
    s = s_ptr.copy()
    d = s - z
    
    ghb_flow = -mf6.get_value(ghb_flow_tag) * cfd2cms
    ghb_depth = ghb_flow / cell_area
    
    sfincs_qexternal = ghb2qext.dot(ghb_depth)
    sfincs_qexternal[d == 0.0] = 0.0
    qexchange[:] = sfincs_qexternal[:]

    # print("min/max qexchange: ", qexchange.min(), qexchange.max())

    # update results dictionaries
    qext_dict[key] = sfincs_qexternal[:].copy()

#### Run each time step

Control loop may be specific for each coastal model.

In [39]:
print(
    f"Coastal model current_time: {coastal.get_current_time():15,.1f} sec. ({coastal.get_current_time()/86400.:15,.1f} days)\n"
     + f"Coastal model end_time:     {coastal.get_end_time():15,.1f} sec. ({coastal.get_end_time()/86400.:15,.1f} days)"
)

Coastal model current_time:             0.0 sec. (            0.0 days)
Coastal model end_time:         7,776,000.0 sec. (           90.0 days)


In [40]:
coastal_coupling_times = np.arange(mf_couple_freq, coastal.get_end_time() + mf_couple_freq, mf_couple_freq)
coastal_coupling_times.shape, coastal_coupling_times[0], coastal_coupling_times[-1]

((2160,), 3600.0, 7776000.0)

In [41]:
# specific to sfincs
t0 = time.perf_counter()
jdx = 0
for to_time in coastal_coupling_times:
    coastal.update_until(to_time)

    mf6.prepare_time_step(mf6.get_time_step())
    update_mf(str(jdx), zs, zb)
    mf6.do_time_step()
    mf6.finalize_time_step()

    update_coastal(str(jdx), coastal_dtuser, zs, zb)

    jdx += 1

t1 = time.perf_counter()
print(f"\nrun time: {(t1 - t0) / 60.} min")

   0% complete,       - s remaining ...
   5% complete,   461.4 s remaining ...
  10% complete,   423.1 s remaining ...
  15% complete,   402.2 s remaining ...
  20% complete,   379.2 s remaining ...
  25% complete,   358.7 s remaining ...
  30% complete,   334.5 s remaining ...
  35% complete,   309.9 s remaining ...
  40% complete,   287.2 s remaining ...
  45% complete,   263.5 s remaining ...
  50% complete,   239.9 s remaining ...
  55% complete,   215.8 s remaining ...
  60% complete,   191.6 s remaining ...
  65% complete,   167.8 s remaining ...
  70% complete,   143.6 s remaining ...
  75% complete,   119.5 s remaining ...
  80% complete,    95.5 s remaining ...
  85% complete,    71.6 s remaining ...
  90% complete,    47.7 s remaining ...
  95% complete,    23.9 s remaining ...
 100% complete,     0.0 s remaining ...

run time: 7.940743731266665 min


In [42]:
# Finalize
mf6.finalize()

In [43]:
coastal.finalize()


 ---------- Simulation finished ----------

 Total time             :    476.845
 Total simulation time  :    476.744
 Time in input          :      0.101
 Time in boundaries     :      2.162 (  0.5%)
 Time in momentum       :    290.902 ( 61.0%)
 Time in continuity     :     64.053 ( 13.4%)
 Time in output         :     90.920 ( 19.1%)

 Average time step (s)  :      6.301

 ---------- Closing off SFINCS -----------


#### Save ghb elevation and conductance data to compressed files

In [44]:
np.savez_compressed(f"{mf_run_path}/ghb_elev.npz", **elev_dict)
np.savez_compressed(f"{mf_run_path}/ghb_cond.npz", **cond_dict)
np.savez_compressed(f"{mf_run_path}/qext.npz", **qext_dict)